<a href="https://colab.research.google.com/github/DinurakshanRavichandran/Visio-Glance/blob/Pre-Processed-Datasets-NLP/dmePPFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from imblearn.over_sampling import SMOTE
import re

# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/DSGP PROJECT 29/DATASETS/Synthetic_DME_Dataset.csv')

# Step 1: Handle Missing Values
# Check for missing values and replace them if any
df.fillna(df.mode().iloc[0], inplace=True)

# Step 2: Feature Engineering
# For this dataset, feature engineering might not be extensive without specific domain insights.
# An example could be deriving categorizations or groups from existing data if necessary.

# Step 3: Text Processing
# Clean and process the 'Visual Symptoms' text data
def clean_text(text):
    text = text.lower()  # convert to lower case
    text = re.sub(r'\W', ' ', text)  # remove all non-word characters
    text = re.sub(r'\s+', ' ', text, flags=re.I)  # replace multiple spaces with a single space
    return text.strip()

df['Visual Symptoms'] = df['Visual Symptoms'].apply(clean_text)

# Step 4: Encode Categorical Data
# Using LabelEncoder for categorical features
label_encoders = {}
for column in ['Visual Acuity Test Results', 'Lens Status', 'Visual Symptoms']:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le

# Encode the target variable 'Diagnosis'
target_encoder = LabelEncoder()
df['Diagnosis'] = target_encoder.fit_transform(df['Diagnosis'])

# Step 5: Normalize/Scale Numerical Data
# Normalizing the 'Age' and 'Intraocular Pressure (IOP)' columns
scaler = MinMaxScaler()
df[['Age', 'Intraocular Pressure (IOP)']] = scaler.fit_transform(df[['Age', 'Intraocular Pressure (IOP)']])

# Step 6: Handle Class Imbalance
# Using SMOTE to handle class imbalance in the 'Diagnosis' column
X = df.drop('Diagnosis', axis=1)
y = df['Diagnosis']
smote = SMOTE()
X_res, y_res = smote.fit_resample(X, y)

# Reconstruct the DataFrame after resampling
df_resampled = pd.DataFrame(X_res, columns=X.columns)
df_resampled['Diagnosis'] = y_res

# Step 7: Save Preprocessed Data
# Save the final preprocessed data to a new CSV file
preprocessed_file_path = '/content/drive/MyDrive/DSGP PROJECT 29/FINAL MODEL/Preprocessed_DME_Dataset.csv'
df_resampled.to_csv(preprocessed_file_path, index=False)

preprocessed_file_path

#print head of pre processed dataset.
df_resampled.head()

,Age,Visual Acuity Test Results,Intraocular Pressure (IOP),Lens Status,Visual Symptoms,Diagnosis
0,0.644068,5,1.000000,2,244,0
1,0.864407,2,0.958333,4,130,0
2,0.474576,1,0.583333,2,93,0
3,0.237288,0,0.458333,2,180,0
4,0.711864,2,0.916667,2,300,1
